In [1]:
# FN_Notebook_YourName_Group_ITAI2376
# ScholarBot - Simple Research Assistant Agent (Fake Search Version)

# Cell 1: Imports and basic setup

from dataclasses import dataclass, field
from typing import List, Dict, Any
import textwrap
import statistics

In [2]:
# Cell 2: Fake "web search" knowledge base and tools

# A small built-in knowledge base ScholarBot will pretend to "search"
FAKE_KNOWLEDGE_BASE = [
    {
        "topic": "renewable energy",
        "keywords": ["renewable", "solar", "wind", "energy"],
        "snippet": (
            "Global renewable energy capacity has been growing steadily, "
            "driven mainly by falling costs in solar and wind power. "
            "Many countries now set targets for net-zero emissions by mid-century."
        ),
        "source": "Example Report on Renewable Energy Trends"
    },
    {
        "topic": "ai ethics in medicine",
        "keywords": ["ai", "ethics", "medicine", "healthcare", "medical"],
        "snippet": (
            "AI ethics in medicine focuses on fairness, transparency, "
            "accountability, and patient privacy. Models can amplify biases "
            "in training data if not carefully evaluated."
        ),
        "source": "Example Article on AI Ethics in Healthcare"
    },
    {
        "topic": "cybersecurity in education",
        "keywords": ["cybersecurity", "schools", "education", "ransomware"],
        "snippet": (
            "Schools and universities face increasing cyber attacks, including "
            "phishing, ransomware, and data breaches targeting student records. "
            "Security education and basic hygiene like strong passwords remain weak points."
        ),
        "source": "Example Brief on Cybersecurity in Education"
    },
    {
        "topic": "large language models",
        "keywords": ["llm", "language model", "gpt", "chatbot"],
        "snippet": (
            "Large language models are trained on massive text corpora to predict "
            "the next token in a sequence. They can assist with writing, coding, "
            "and reasoning tasks but can also hallucinate incorrect facts."
        ),
        "source": "Example Overview of Large Language Models"
    }
]


def FakeSearchTool(query: str) -> Dict[str, Any]:
    """
    Fake search tool.
    - Takes a query string
    - Returns a list of "results" from the local FAKE_KNOWLEDGE_BASE
    This satisfies the tool integration requirement without using the real internet.
    """

    q = query.lower()
    hits = []

    for entry in FAKE_KNOWLEDGE_BASE:
        # Simple matching: if any keyword from the entry is in the query, consider it a hit
        for kw in entry["keywords"]:
            if kw in q:
                hits.append({
                    "topic": entry["topic"],
                    "snippet": entry["snippet"],
                    "source": entry["source"]
                })
                break  # avoid duplicate from same entry

    # If nothing matched, return an empty list
    return {"results": hits}


def SummarizerTool(query: str, results: List[Dict[str, str]]) -> str:
    """
    Simple summarizer tool.
    Uses templates and basic string handling to create a short summary.
    No external models. This is our second "tool" for the assignment.
    """
    if not results:
        return (
            f"I could not find specific information related to '{query}'. "
            "You may need more specialized sources or a broader topic."
        )

    # Take up to 2-3 results and compress them into a small summary
    bullets = []
    for r in results[:3]:
        bullets.append(f"- {r['snippet']}")

    summary = f"Here is a short summary for: '{query}':\n"
    summary += "\n".join(bullets)
    return summary

In [3]:
# Cell 3: Simple memory and feedback system

@dataclass
class InteractionRecord:
    query: str
    summary: str
    sources: List[str]
    rating: int = None  # user rating from 1 to 5, optional


@dataclass
class ScholarMemory:
    interactions: List[InteractionRecord] = field(default_factory=list)

    def add_interaction(self, record: InteractionRecord):
        self.interactions.append(record)

    def update_rating(self, query: str, rating: int):
        # update the rating for the most recent matching query
        for record in reversed(self.interactions):
            if record.query == query:
                record.rating = rating
                return True
        return False

    def average_rating(self) -> float:
        ratings = [r.rating for r in self.interactions if r.rating is not None]
        if not ratings:
            return 0.0
        return statistics.mean(ratings)

In [25]:
# Cell 4: ScholarBot agent definition

class ScholarBotAgent:
    """
    ScholarBot - a simple research assistant agent.
    - Uses FakeSearchTool to "search" a local KB
    - Uses SummarizerTool to summarize findings
    - Stores past interactions and ratings
    - Implements safety checks and ReAct-style reasoning
    """

    def __init__(self):
        self.memory = ScholarMemory()

    # -----------------------------
    # 1. Safety and input validation
    # -----------------------------
    def _is_safe(self, query: str) -> bool:
        blocked_keywords = ["harm", "kill", "weapon", "attack"]
        q = query.lower()
        return not any(bad in q for bad in blocked_keywords)

    # -----------------------------
    # 2. Chain-of-thought reasoning
    # -----------------------------
    def _reason_about_query(self, query: str) -> str:
        return (
            "I will identify the main topic, use the fake search tool to gather "
            "relevant information, then summarize the key points."
        )

    # -----------------------------
    # 3. Main Research Method
    # -----------------------------
    def research(self, query: str) -> Dict[str, Any]:

        # Safety
        if not self._is_safe(query):
            return {
                "query": query,
                "reasoning": "Query blocked by safety filter.",
                "summary": (
                    "This request involves restricted topics. "
                    "ScholarBot cannot continue."
                ),
                "results": [],
                "sources": []
            }

        # Reasoning trace for transparency
        reasoning = self._reason_about_query(query)

        # Tool 1 — Fake Search
        search_output = FakeSearchTool(query)
        results = search_output.get("results", [])

        # Tool 2 — Summarizer
        summary = SummarizerTool(query, results)

        # Sources
        sources = list({r.get("source", "unknown") for r in results})

        # Memory record
        record = InteractionRecord(
            query=query,
            summary=summary,
            sources=sources
        )
        self.memory.add_interaction(record)

        # Build a structured return object
        return {
            "query": query,
            "reasoning": reasoning,
            "results": results,
            "summary": summary,
            "sources": sources,
            "current_average_rating": self.memory.average_rating()
        }

    # -----------------------------
    # 4. RL Feedback
    # -----------------------------
    def give_feedback(self, query: str, rating: int) -> str:
        if rating < 1 or rating > 5:
            return "Rating must be between 1 and 5."

        updated = self.memory.update_rating(query, rating)
        if not updated:
            return "No past interaction found for that query."

        avg = self.memory.average_rating()
        return f"Thanks! Current average rating: {avg:.2f}"

    # -----------------------------
    # 5. History viewer
    # -----------------------------
    def show_history(self, limit: int = 5) -> str:
        last = self.memory.interactions[-limit:]
        if not last:
            return "No history yet."

        lines = []
        for idx, r in enumerate(last, 1):
            lines.append(f"{idx}. Query: {r.query}")
            lines.append(f"   Rating: {r.rating or 'None yet'}")
            lines.append(f"   Sources: {', '.join(r.sources)}")
        return "\n".join(lines)

    # -----------------------------
    # 6. UNIFIED RUN METHOD
    # -----------------------------
    def run(self, query: str) -> str:
        """
        Wrapper used by the demo + chat cells.
        Turns the structured output of research() into displayable text.
        """
        output = self.research(query)

        text = []
        text.append(f"🔍 ScholarBot Researching: {output['query']}\n")
        text.append(f"🤔 Reasoning: {output['reasoning']}\n")

        # Results
        if output["results"]:
            text.append("📘 Top Results:")
            for r in output["results"]:
                title = r.get("title", "Untitled Result")
                snippet = r.get("snippet", "No snippet available")
                text.append(f" • {title}: {snippet}")
            text.append("")

        # Summary
        text.append(f"📝 Summary:\n{output['summary']}\n")

        # Sources
        if output["sources"]:
            text.append("📚 Sources Used:")
            for s in output["sources"]:
                text.append(f" • {s}")
            text.append("")

        # Rating
        avg = output["current_average_rating"]
        text.append(f"📊 Current Average Rating: {avg:.2f}")

        return "\n".join(text)


In [26]:
# Cell 5: Helper to pretty print ScholarBot responses

def print_response(resp: Dict[str, Any]):
    print(f"🔍 Query: {resp['query']}\n")
    print("🤔 Reasoning:")
    print(textwrap.fill(resp["reasoning"], width=80))
    print("\n📚 Sources:")
    if resp["sources"]:
        for s in resp["sources"]:
            print(f" - {s}")
    else:
        print(" - None")

    print("\n📝 Summary:")
    print(textwrap.fill(resp["summary"], width=80))

    print(f"\n📊 Current Average Rating: {resp['current_average_rating']:.2f}")

In [27]:
# Cell 6: Create ScholarBot and run demo queries

agent = ScholarBotAgent()

queries = [
    "What are current renewable energy trends?",
    "Explain AI ethics in medicine in simple terms.",
    "What are some cybersecurity risks in education?",
    "Tell me about large language models."
]

for q in queries:
    print("=" * 80)
    resp = agent.research(q)
    print_response(resp)
    print("\n")

🔍 Query: What are current renewable energy trends?

🤔 Reasoning:
I will identify the main topic, use the fake search tool to gather relevant
information, then summarize the key points.

📚 Sources:
 - Example Report on Renewable Energy Trends

📝 Summary:
Here is a short summary for: 'What are current renewable energy trends?': -
Global renewable energy capacity has been growing steadily, driven mainly by
falling costs in solar and wind power. Many countries now set targets for net-
zero emissions by mid-century.

📊 Current Average Rating: 0.00


🔍 Query: Explain AI ethics in medicine in simple terms.

🤔 Reasoning:
I will identify the main topic, use the fake search tool to gather relevant
information, then summarize the key points.

📚 Sources:
 - Example Article on AI Ethics in Healthcare

📝 Summary:
Here is a short summary for: 'Explain AI ethics in medicine in simple terms.': -
AI ethics in medicine focuses on fairness, transparency, accountability, and
patient privacy. Models can amp

In [28]:
# Cell 7: Provide feedback and inspect simple reinforcement behavior

# Give feedback to a couple of queries
print(agent.give_feedback("What are current renewable energy trends?", 4))
print(agent.give_feedback("Explain AI ethics in medicine in simple terms.", 5))

print("\nHistory (most recent interactions):")
print(agent.show_history(limit=5))


Thanks! Current average rating: 4.00
Thanks! Current average rating: 4.50

History (most recent interactions):
1. Query: What are current renewable energy trends?
   Rating: 4
   Sources: Example Report on Renewable Energy Trends
2. Query: Explain AI ethics in medicine in simple terms.
   Rating: 5
   Sources: Example Article on AI Ethics in Healthcare
3. Query: What are some cybersecurity risks in education?
   Rating: None yet
   Sources: Example Brief on Cybersecurity in Education
4. Query: Tell me about large language models.
   Rating: None yet
   Sources: Example Overview of Large Language Models


In [29]:
def scholarbot_demo():
    print("===== SCHOLARBOT DEMONSTRATION =====\n")

    demo_queries = [
        "renewable energy trends",
        "AI ethics in medicine",
        "cybersecurity in education"
    ]

    for q in demo_queries:
        print(f"\n=== QUERY: {q.upper()} ===\n")

        # Call agent
        response = agent.run(q)
        print(response)

        # Give automatic RL rating
        rating = 5
        print(f"\n👍 Giving Feedback Rating: {rating}")
        fb = agent.give_feedback(q, rating)
        print(fb)
        print("\n" + "-"*50)

    print("\n===== END OF DEMO =====\n")

# Run the demo
scholarbot_demo()


===== SCHOLARBOT DEMONSTRATION =====


=== QUERY: RENEWABLE ENERGY TRENDS ===

🔍 ScholarBot Researching: renewable energy trends

🤔 Reasoning: I will identify the main topic, use the fake search tool to gather relevant information, then summarize the key points.

📘 Top Results:
 • Untitled Result: Global renewable energy capacity has been growing steadily, driven mainly by falling costs in solar and wind power. Many countries now set targets for net-zero emissions by mid-century.

📝 Summary:
Here is a short summary for: 'renewable energy trends':
- Global renewable energy capacity has been growing steadily, driven mainly by falling costs in solar and wind power. Many countries now set targets for net-zero emissions by mid-century.

📚 Sources Used:
 • Example Report on Renewable Energy Trends

📊 Current Average Rating: 4.50

👍 Giving Feedback Rating: 5
Thanks! Current average rating: 4.67

--------------------------------------------------

=== QUERY: AI ETHICS IN MEDICINE ===

🔍 Schola